In [ ]:
import cv2
import numpy as np

#实现图像的校正
def do_Image_correction(images): 
    #获取图像
    im_src = images[0]
    #获取进行变换的区域的四个角点 
    pts_src = images[1]
    
    #获取目标图像变换大小及变换后的对应四个角点的位置
    dst_w, dst_h = images[2], images[3]
    pts_dst = np.array([[0, 0],[dst_w-1 , 0],[dst_w-1, dst_h-1],[0, dst_h-1]])  
 
    # 计算单应性矩阵 Homography
    h, status = cv2.findHomography(pts_src, pts_dst)
    print("图像校正单应性矩阵：")
    print(h)

    # 应用单应性矩阵进行透视变换
    im_out = cv2.warpPerspective(im_src, h, (dst_w, dst_h))
    return im_out

#实现两张图像的拼接
def do_Image_Stitch(img1, img2):
    # 将彩色图像转换为灰度图
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    # 创建SIFT检测器
    sift = cv2.SIFT_create()

    # 检测关键点和描述符
    kp1, des1 = sift.detectAndCompute(gray1, None)
    kp2, des2 = sift.detectAndCompute(gray2, None)

    # 匹配描述符
    matcher = cv2.BFMatcher()
    matches = matcher.knnMatch(des1, des2, k=2)

    # 应用比率测试
    good_matches = []
    for m,n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    # 计算变换矩阵
    if len(good_matches) > 4:
        src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

        matrix, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        
        # 计算输出图像的尺寸
        h1, w1 = img1.shape[:2]
        h2, w2 = img2.shape[:2]
        points1 = np.float32([[0, 0], [0, h1], [w1, h1], [w1, 0]]).reshape(-1, 1, 2)
        points2 = np.float32([[0, 0], [0, h2], [w2, h2], [w2, 0]]).reshape(-1, 1, 2)
        points2_ = cv2.perspectiveTransform(points1, matrix)
        points = np.concatenate((points2, points2_), axis=0)
        [x_min, y_min] = np.int32(points.min(axis=0).ravel() - 0.5)
        [x_max, y_max] = np.int32(points.max(axis=0).ravel() + 0.5)
        translation_dist = [-x_min, -y_min]
        H_translation = np.array([[1, 0, translation_dist[0]], [0, 1, translation_dist[1]], [0, 0, 1]])

        # 用变换矩阵变换第一张图
        output_img = cv2.warpPerspective(img1, H_translation.dot(matrix), (x_max-x_min, y_max-y_min))
        output_img[translation_dist[1]:translation_dist[1]+h2, translation_dist[0]:translation_dist[0]+w2] = img2

        return output_img
    else:
        return None


#将图像列表中图像逐一拼接
def q_stitch_images(images):
    current_pano = images[0]
    for i in range(1, len(images)):
        current_pano = do_Image_Stitch(current_pano, images[i])
        if current_pano is None:
            raise Exception("Error in stitching images at index {}".format(i))
    return current_pano


def main():
#1
    # 加载用于图像校正的图像,四个角点位置
    img_src = cv2.imread('/home/kh/homework/exercise4/img/Img_Correction/book_src.jpg')  #book 
    pts_src = np.array([[233, 773], [747, 775], [1101, 1421],[188, 1565]])  
    images_1 = [img_src, pts_src, 1280, 1706]
    # img_src = cv2.imread('/home/kh/homework/exercise4/img/Img_Correction/license_plate_src.jpg')  #license plate
    # pts_src = np.array([[878, 549], [1018, 714], [972, 759],[829, 595]])  
    # images_1 = [img_src, pts_src, 300, 100]
    
    # 加载用于图像拼接的图像
    img1 = cv2.imread('/home/kh/homework/exercise4/img/Img_Stitch/test1_1.jpg')
    img2 = cv2.imread('/home/kh/homework/exercise4/img/Img_Stitch/test1_2.jpg')
    image_name = 'test1' + ".jpg" #图像命名：时间戳.jpg 
    images_2 = [img1,img2]    
    # img1 = cv2.imread('/home/kh/homework/exercise4/img/Img_Stitch/test2_1.jpg')
    # img2 = cv2.imread('/home/kh/homework/exercise4/img/Img_Stitch/test2_2.jpg')
    # img3 = cv2.imread('/home/kh/homework/exercise4/img/Img_Stitch/test2_3.jpg')
    # image_name = 'test2' + ".jpg" #图像命名：时间戳.jpg 
    # images_2 = [img1,img2,img3]
 
#2、
    #单性变换--图像校正实例运用
    img_out = do_Image_correction(images_1)
    cv2.imwrite("/home/kh/homework/exercise4/img/Img_Correction/book_res.jpg", img_out)  ##book 
    # cv2.imwrite("/home/kh/homework/exercise4/img/Img_Correction/license_plate_res.jpg", img_out)  #license plate
    
    #单性变换--图像拼接实例运用
    result = q_stitch_images(images_2)
    if result is not None:    
        cv2.imwrite( '/home/kh/homework/exercise4/img/Img_Stitch/' + image_name, result)  #保存；
        print("图像拼接成功！")
    else:
        print("图像拼接失败！") 

if __name__ =="__main__":     
    main()